## Import

In [ ]:
# Import of basic packages
import numpy as np
import pandas as pd
import operator
from joblib import dump, load
import sys
import warnings

warnings.filterwarnings('ignore')

# Import of chart packages
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import altair as alt

# Import for normal distribution
import pylab
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import shapiro

# Import of machine learning metric packages
from sklearn.metrics import make_scorer, f1_score, classification_report, confusion_matrix, mean_squared_error, r2_score, accuracy_score, recall_score, precision_score, roc_auc_score, roc_curve, fbeta_score
from sklearn import metrics
from scipy.stats import randint, uniform, loguniform

# Import of preprossesor packages
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, LabelBinarizer, PolynomialFeatures

# Import of machine learning packages
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, RandomForestClassifier, RandomForestRegressor, VotingClassifier, StackingRegressor, StackingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

# Set random seed 
RSEED = 0

## Style

In [ ]:
pd.set_option('display.max_columns', None)

## Load

In [ ]:
# Load csv
df = pd.read_csv('data/train.csv')
df.head(3)

## Explore data analysis

In [ ]:
# EDA info & describe
info = pd.concat([
df.dtypes.to_frame().T,
df.mean(numeric_only=True).to_frame().T,
df.std(numeric_only=True).to_frame().T,
df.min(numeric_only=True).to_frame().T,
df.quantile(0.25, numeric_only=True).to_frame().T,
df.quantile(0.5, numeric_only=True).to_frame().T, 
df.quantile(0.75, numeric_only=True).to_frame().T,
df.max(numeric_only=True).to_frame().T,], ignore_index=True).applymap(lambda x: round(x, 1) if isinstance(x, (int, float)) else x)

info.insert(0, 'statistic', ['dtype', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])
info

In [ ]:
# EDA duplicates
duplicates = df.duplicated().sum()
duplicate_percentage = round((duplicates / df.shape[0]) * 100, 1)
df[df.duplicated(keep=False)].head(10)

In [ ]:
# EDA NaNs
missing = pd.DataFrame(df.isnull().sum(), columns=['Amount'])
missing['Percentage'] = round((missing['Amount']/df.shape[0]) * 100, 1)
missing[missing['Amount'] != 0]

In [ ]:
# EDA shape
print('Number of rows and columns: ',df.shape)
print('-'*50)
pd.concat([df.head(3), df.tail(3)]).reset_index(drop=True)

In [ ]:
# EDA Uniques
unique_counts = pd.DataFrame(df.nunique(), columns=['Amount']).sort_values('Amount', ascending=False).T
unique_counts

In [ ]:
df.head(10)

In [ ]:
# Get dataframe for delayed flights
df_delayed = df.copy()
df_delayed = df_delayed[df_delayed["target"] > 0]
df_delayed.info()

In [ ]:
# Overview of target distribution
print("General statistics of target:\n", (df['target']).describe())

# Creating a figure composed of two matplotlib.Axes objects (ax_box and ax_hist)
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)})
 
# Assigning a graph to each ax
sns.boxplot(df["target"], orient="h", ax=ax_box, color='lightblue')
sns.histplot(data=df, x="target", ax=ax_hist, color='lightblue')

# Remove x axis name for the boxplot
ax_box.set(xlabel='')
ax_box.set_xlim([-20, 500])
plt.show()

In [ ]:
# Overview of target distribution (of delayed flights)
print("General statistics of target (of delayed flights):\n", (df_delayed['target']).describe())

# Creating a figure composed of two matplotlib.Axes objects (ax_box and ax_hist)
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)})
 
# Assigning a graph to each ax
sns.boxplot(df_delayed["target"], orient="h", ax=ax_box, color='lightblue')
sns.histplot(data=df_delayed, x="target", ax=ax_hist, color='lightblue')

# Remove x axis name for the boxplot
ax_box.set(xlabel='')
ax_box.set_xlim([-20, 500])
plt.show()

In [ ]:
stat, p = shapiro(df_delayed['target'])

# Interpretation of p-value
alpha = 0.05
if p > alpha:
    print('The data of column "target" looks normally distributed (p > 0.05, fail to reject H0).')
else:
    print('The data of column "target" does not look normally distributed (p < 0.05, reject H0).')

In [ ]:
# Grafic methode: Q-Q-Plot
stats.probplot(df_delayed['target'], dist="norm", plot=pylab)
pylab.show()

Check influence of departure and arrival airport route

In [ ]:
df['flight_route'] = df["DEPSTN"] + df["ARRSTN"]

In [ ]:
# Get 5 most frequent flight routes
df['flight_route'].value_counts(sort=True).head(5)

In [ ]:
df[df["flight_route"]=="TUNTUN"].head(5)

In [ ]:
# Get all entries, where dep and arr airport are the same
df[df["DEPSTN"]==df["ARRSTN"]].shape

In [ ]:
# Start with a basic boxplot
sns.boxplot(x="flight_route", y="target", data=df[(df['flight_route']=="TUNORY") | (df['flight_route']=="ORYTUN") | (df['flight_route']=="TUNTUN")])

# add title
plt.title("Flight delay time for 3 most frequent flight routes")
plt.ylim(-5, 140)

# show the graph
plt.show()

In [ ]:
# Get top 5 flight routes with most delay time
top_categories = df.groupby('flight_route')['target'].sum().nlargest(5)

print(top_categories)

In [ ]:
print("ORYTUN contribution to total delay time in %: ", round((487348/df["target"].sum())*100, 2))

In [ ]:
# Start with a basic boxplot
sns.boxplot(x="flight_route", y="target", data=df[(df['flight_route']=="ORYTUN") | (df['flight_route']=="TUNORY") | (df['flight_route']=="ISTTUN")])

# add title
plt.title("Top 3 flight routes with most delay time")
plt.ylim(-5, 140)

# show the graph
plt.show()

In [ ]:
df = df.drop("flight_route", axis=1)

## Data cleaning

In [ ]:
# Clean column names
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns.str.lower()

Feature 'ac' holds information about the model of the airplane. Extracting and converting the information.

In [ ]:
df['airplane_model'] = df['ac'].str[3:6]
df.head()

In [ ]:
# EDA Uniques
unique_counts = pd.DataFrame(df.nunique(), columns=['Amount']).sort_values('Amount', ascending=False).T
unique_counts

In [ ]:
set(df['airplane_model'])

In [ ]:
replacement_dict = {
    '31A': 'Airbus',
    '31B': 'Airbus',
    '320': 'Airbus',
    '321': 'Airbus',
    '32A': 'Airbus',
    '332': 'Airbus',
    '343': 'Airbus',
    '345': 'Airbus',
    '733': 'Boeing',
    '734': 'Boeing',
    '736': 'Boeing',
    'AT7': 'ATR',
    'CR9': 'Bombardier'
}

df['producer'] = df['airplane_model']

# Replace values in the 'purpose' column
df['producer'] = df['producer'].replace(replacement_dict)


In [ ]:
df.head()

In [ ]:
df['airline_1'] = df['fltid'].str[0:2]
df.head()

In [ ]:
set(df['airline_1'])

In [ ]:
df['airline_2'] = df['ac'].str[0:2]
df.head()

In [ ]:
set(df['airline_2'])

In [ ]:
df.head()

Load airpots data set and clean. 

Merge data sets based on airport short handle. 

In [ ]:
# Add columns with weekdays, yyyy, mm, dd, hh:mm:ss

y = '_year'
m = '_month'
wd = '_wd'
M = '_min'

### std ###

date = 'std'

idx = df.columns.get_loc(date)

df[date] = pd.to_datetime(df[date], format='%Y-%m-%d %H:%M:%S')
df.insert(loc=idx+1, column=date+y, value=df[date].dt.strftime('%Y')) # year yyyy
df.insert(loc=idx+2, column=date+m, value=df[date].dt.strftime('%#m')) # month m
df.insert(loc=idx+3, column=date+wd, value=df[date].dt.strftime('%w')) # weekday wd
h = df[date].dt.strftime('%#H').astype(int) # hours
minutes = df[date].dt.strftime('%#M').astype(int) # minutes
# calcualte time in just minutes
t = 60*h + minutes
df.insert(loc=idx+4, column=date+M, value=t) # minutes

### sta ###

date = 'sta'

idx = df.columns.get_loc(date)

df[date] = pd.to_datetime(df[date], format='%Y-%m-%d %H.%M.%S')
df.insert(loc=idx+1, column=date+y, value=df[date].dt.strftime('%Y')) # year yyyy
df.insert(loc=idx+2, column=date+m, value=df[date].dt.strftime('%#m')) # month m
df.insert(loc=idx+3, column=date+wd, value=df[date].dt.strftime('%w')) # weekday wd
h = df[date].dt.strftime('%#H').astype(int) # hours
minutes = df[date].dt.strftime('%#M').astype(int)
# calcualte time in just minutes
t = 60*h + minutes
df.insert(loc=idx+4, column=date+M, value=t) # minutes

### datop ###

date = 'datop'

idx = df.columns.get_loc(date)

df[date] = pd.to_datetime(df[date], format='%Y-%m-%d')
df.insert(loc=idx+1, column=date+y, value=df[date].dt.strftime('%Y')) # year yyyy
df.insert(loc=idx+2, column=date+m, value=df[date].dt.strftime('%#m')) # month m
df.insert(loc=idx+3, column=date+wd, value=df[date].dt.strftime('%w')) # weekday wd

# convert new columns as integers
list = ['std_year', 'std_month', 'std_wd', 'sta_year', 'sta_month', 'sta_wd', 'datop_year', 'datop_month', 'datop_wd', 'target']

for date in list:
    df[date] = df[date].astype(int)

# change weekday numbers to EU where day 1 = Monday
list = ['std_wd', 'sta_wd', 'datop_wd']

for date in list:
    df[date][df[date] == 0] = 7 # Sunday

Geo-encoding of airports

In [ ]:
# Load csv
df_airports = pd.read_csv('data/airports.csv')
df_airports.columns = ['id', 'name', 'city', 'country', 'short', 'rubbish_6', 'latitude', 'longitude', 'rubbish_1', 'rubbish_2', 'rubbish_3', 'rubbish_4', 'type', 'rubbish_5']
df_airports = df_airports.drop(['id', 'name', 'rubbish_1', 'rubbish_2', 'rubbish_3', 'rubbish_4', 'rubbish_5', 'rubbish_6', 'type'], axis=1)
df_airports = df_airports.dropna(subset=['short'])
df_airports.head(3)


In [ ]:
df = df.merge(df_airports, left_on='depstn', right_on='short', how='left', suffixes=('', '_dep'))

# Merge based on arrival station
df = df.merge(df_airports, left_on='arrstn', right_on='short', how='left', suffixes=('', '_arr'))

# Rename columns for clarity
df = df.rename(columns={
    'city': 'city_dep',
    'country': 'country_dep',
    'latitude': 'latitude_dep',
    'longitude': 'longitude_dep'
})

df.head(3)


In [ ]:
df = df.drop_duplicates(subset='id', keep='first')
df.shape

In [ ]:
(df[(df["datop_year"]==df["std_year"])&(df["datop_month"]==df["std_month"])&(df["datop_wd"]==df["std_wd"])]).shape

In [ ]:
# EDA info & describe
info = pd.concat([
df.dtypes.to_frame().T,
df.mean(numeric_only=True).to_frame().T,
df.std(numeric_only=True).to_frame().T,
df.min(numeric_only=True).to_frame().T,
df.quantile(0.25, numeric_only=True).to_frame().T,
df.quantile(0.5, numeric_only=True).to_frame().T, 
df.quantile(0.75, numeric_only=True).to_frame().T,
df.max(numeric_only=True).to_frame().T,], ignore_index=True).applymap(lambda x: round(x, 1) if isinstance(x, (int, float)) else x)

info.insert(0, 'statistic', ['dtype', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])
info

In [ ]:
df.head(3)

In [ ]:
# Your existing code to create the scatter plot
fig = px.scatter_mapbox(df, lat="latitude_arr", lon="longitude_arr", 
                        color_discrete_sequence=["teal"], zoom=3)

# Custom mapbox style with opacity
mapbox_style = {
    "version": 8,
    "sources": {
        "osm": {
            "type": "raster",
            "tiles": ["https://a.tile.openstreetmap.org/{z}/{x}/{y}.png"],
            "tileSize": 256,
            "attribution": "&copy; OpenStreetMap Contributors",
            "maxzoom": 19
        }
    },
    "layers": [{
        "id": "osm",
        "type": "raster",
        "source": "osm",
        "paint": {"raster-opacity": 0.6}  # Set the opacity here
    }]
}

# Update layout with custom mapbox style
fig.update_layout(mapbox_style=mapbox_style)

# Your existing layout updates
fig.update_layout(width=1200, height=800)
fig.update_layout(title="Distribution of airports", title_x=0.5)

fig.show()

In [ ]:
df.to_csv('data/data.csv', index=False)

## Feature engineering

In [ ]:
df = df.drop(['datop_year', 'datop_month', 'datop_wd', 'datop', 'fltid', 'std', 'sta', 'ac', 'short', 'short_arr', 'city_dep', 'country_dep', 'city_arr', 'country_arr', 'airline_2', 'producer'], axis=1)
df.head(3)

In [ ]:
# List of columns to encode
columns_to_encode = ['depstn', 'status', 'arrstn', 'airline_1', 'airplane_model'] # reduced by aggressive feature drop

# Create a copy of the original dataframe
df_encoded = df.copy()

# Encode each column separately
for column in columns_to_encode:
    lb = LabelBinarizer()
    encoded = lb.fit_transform(df[column])
    
    # If binary classification, create a single column
    if len(lb.classes_) == 2:
        df_encoded[f'{column}_encoded'] = encoded
    else:
        # For multiclass, create multiple columns
        encoded_df = pd.DataFrame(encoded, columns=[f'{column}_{cls}' for cls in lb.classes_], index=df.index)
        df_encoded = pd.concat([df_encoded, encoded_df], axis=1)

df_encoded = df_encoded.drop(column, axis=1)

# Now, combine the non-encoded columns from df with the encoded columns from df_encoded
df = pd.concat([df, df_encoded], axis=1)


In [ ]:
df = df.drop(['depstn', 'arrstn', 'status', 'airline_1', 'airplane_model'], axis=1) # reduced by aggressive feature drop
duplicate_columns = df.columns[df.columns.duplicated()]
df = df.loc[:, ~df.columns.duplicated()]

##### Target engineering

In [ ]:
# Convert target into certain category intervals

def target_interval(row):
    if row['target'] == 0:
        return 1
    elif 0 < row['target'] <= 30:
        return 2
    elif 30 < row['target'] <= 60:
        return 3
    elif 60 < row['target'] <= 120:
        return 4
    elif 120 < row['target'] <= 240:
        return 5   
    else:
        return 6  
    
df['target_cat'] = df.apply(target_interval, axis=1)

In [ ]:
sns.histplot(data=df['target_cat'])

In [ ]:
df.head(3)

##### Standardization

In [ ]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform only the specified columns
columns_to_standardize = ['std_year', 'std_month', 'std_wd', 'std_min', 'sta_year', 'sta_month', 'sta_wd', 'sta_min', 'latitude_dep', 'longitude_dep', 'latitude_arr', 'longitude_arr']  # Replace with your actual column names
df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

In [ ]:
df = df.drop_duplicates(subset=['id'], keep='first')
df.shape

## Splitting data for testing 

In [ ]:
# Define features and target variable (target)
X = df.drop(['target', 'id', 'target_cat'], axis=1)
y = df['target_cat']

# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=RSEED)

##### Interrupting kernel before model training

In [ ]:
sys.exit()

## Trainining the model

#### Baseline model: LogisticRegression

In [ ]:
# Instantiate and train linear regression model
model_0 = LogisticRegression(max_iter=100)
model_0.fit(X_train, y_train) 

In [ ]:
y_pred_train = model_0.predict(X_train)
y_pred_test = model_0.predict(X_test)

In [ ]:
# Save the model
dump(model_0, 'models/model_0.joblib')

In [ ]:
# Load model
model_0 = load('models/model_0.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train = precision_score(y_train, y_pred_train, average='weighted')
precision_test = precision_score(y_test, y_pred_test, average='weighted')
print('Precision (train): ', round(precision_train, 2))
print('Precision (test): ', round(precision_test, 2))

# Calculate Recall
recall_train = recall_score(y_train, y_pred_train, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
print('Recall (train): ', round(recall_train, 2))
print('Recall (test): ', round(recall_test, 2))

# Calculate F1 Score
f1_train = f1_score(y_train, y_pred_train, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
print('F1 Score (train): ', round(f1_train, 2))
print('F1 Score (test): ', round(f1_test, 2))

In [ ]:
# Calculate Classification Report
classification_train = classification_report(y_train, y_pred_train)
classification_test = classification_report(y_test, y_pred_test)
print('Classification Report (train): ', classification_train)
print('Classification Report (test): ', classification_test)

#### Define variables for storing the scores

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores = {}

#### Model 1: SGDClassifier

In [ ]:
# Instantiate and train SGDClassifier model
model_1 = SGDClassifier(random_state=RSEED)
model_1.fit(X_train, y_train)

##### RandomizedSearchCV

In [ ]:
# # Define the parameter distribution for random search
# param_dist = {
#     'penalty': ['l2', 'l1', 'elasticnet', None],
#     'alpha': loguniform(1e-2, 1e0),
#     'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
# }

# # Create a base model
# #base_estimator = DecisionTreeClassifier(random_state=RSEED)
# SGD = SGDClassifier(random_state=RSEED)

# # Create a custom scorer (you can change this to other metrics if needed)
# scorer = make_scorer(f1_score)

# # Instantiate RandomizedSearchCV object
# random_search = RandomizedSearchCV(
#     estimator=SGD,
#     param_distributions=param_dist,
#     n_iter=50,  # Reduced number of iterations
#     cv=3,  # Reduced number of cross-validation folds
#     scoring=scorer,
#     random_state=RSEED,
#     n_jobs=-1  # use all available cores
# )

# # Fit RandomizedSearchCV
# random_search.fit(X_train, y_train)

# # Print the best parameters and score
# print("Best parameters:", random_search.best_params_)
# print("Best cross-validation score:", random_search.best_score_)

# # Get the best model
# model_1 = random_search.best_estimator_

In [ ]:
y_pred_train_1 = model_1.predict(X_train)
y_pred_test_1 = model_1.predict(X_test)

In [ ]:
# Save the model
dump(model_1, 'models/model_1.joblib')

In [ ]:
# Load model
model_1 = load('models/model_1.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_1 = precision_score(y_train, y_pred_train_1, average='weighted')
precision_test_1 = precision_score(y_test, y_pred_test_1, average='weighted')
print('Precision (train): ', round(precision_train_1, 2))
print('Precision (test): ', round(precision_test_1, 2))

# Calculate Recall
recall_train_1 = recall_score(y_train, y_pred_train_1, average='weighted')
recall_test_1 = recall_score(y_test, y_pred_test_1, average='weighted')
print('Recall (train): ', round(recall_train_1, 2))
print('Recall (test): ', round(recall_test_1, 2))

# Calculate F1 Score
f1_train_1 = f1_score(y_train, y_pred_train_1, average='weighted')
f1_test_1 = f1_score(y_test, y_pred_test_1, average='weighted')
print('F1 Score (train): ', round(f1_train_1, 2))
print('F1 Score (test): ', round(f1_test_1, 2))

In [ ]:
# Calculate Classification Report
classification_train_1 = classification_report(y_train, y_pred_train_1)
classification_test_1 = classification_report(y_test, y_pred_test_1)
print('Classification Report (train): ', classification_train_1)
print('Classification Report (test): ', classification_test_1)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_1_train'] = round(f1_train_1, 2)
f1_scores['f2_1_test'] = round(f1_test_1, 2)

#### Model 2: KNeighborsClassifier

##### GridSearchCV

param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

Best parameters: {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'uniform'}

In [ ]:
# Instantiate and train KNeighborsClassifier
#model_2 = KNeighborsClassifier(**grid_search.best_params_) # Get best parameters from grid search
model_2 = KNeighborsClassifier(n_neighbors=7, weights='uniform', metric='manhattan')
model_2.fit(X_train, y_train)

In [ ]:
# Definition of hyperparameters
param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# GridSearchCV initiation
grid_search = GridSearchCV(model_2, param_grid, cv=5)

# Search for best parameters
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

In [ ]:
y_pred_train_2 = model_2.predict(X_train)
y_pred_test_2 = model_2.predict(X_test)

In [ ]:
# Save the model
dump(model_2, 'models/model_2.joblib')

In [ ]:
# Load model
model_2 = load('models/model_2.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_2 = precision_score(y_train, y_pred_train_2, average='weighted')
precision_test_2 = precision_score(y_test, y_pred_test_2, average='weighted')
print('Precision (train): ', round(precision_train_2, 2))
print('Precision (test): ', round(precision_test_2, 2))

# Calculate Recall
recall_train_2 = recall_score(y_train, y_pred_train_2, average='weighted')
recall_test_2 = recall_score(y_test, y_pred_test_2, average='weighted')
print('Recall (train): ', round(recall_train_2, 2))
print('Recall (test): ', round(recall_test_2, 2))

# Calculate F1 Score
f1_train_2 = f1_score(y_train, y_pred_train_2, average='weighted')
f1_test_2 = f1_score(y_test, y_pred_test_2, average='weighted')
print('F1 Score (train): ', round(f1_train_2, 2))
print('F1 Score (test): ', round(f1_test_2, 2))

In [ ]:
# Calculate Classification Report
classification_train_2 = classification_report(y_train, y_pred_train_2)
classification_test_2 = classification_report(y_test, y_pred_test_2)
print('Classification Report (train): ', classification_train_2)
print('Classification Report (test): ', classification_test_2)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_2_train'] = round(f1_train_2, 2)
f1_scores['f2_2_test'] = round(f1_test_2, 2)

#### Model 3: Decision Tree Classifier

In [ ]:
# Instantiate and train decision tree classifier on gini
#model_3 = DecisionTreeClassifier(criterion='gini', random_state=RSEED)
#model_3.fit(X_train, y_train)

##### RandomizedSearchCV

In [ ]:
# Define the parameter distribution for random search
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': uniform(0, 1)
}

# Create a base model
dt = DecisionTreeClassifier(random_state=RSEED)

# Create a custom scorer (you can change this to other metrics if needed)
scorer = make_scorer(accuracy_score)

# Instantiate RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=param_dist,
    n_iter=100,  # number of parameter settings that are sampled
    cv=5,  # number of cross-validation folds
    scoring=scorer,
    random_state=RSEED,
    n_jobs=-1  # use all available cores
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Get the best model
model_3 = random_search.best_estimator_

In [ ]:
y_pred_train_3 = model_3.predict(X_train)
y_pred_test_3 = model_3.predict(X_test)

In [ ]:
# Save the model
dump(model_3, 'models/model_3.joblib')

In [ ]:
# Load model
model_3 = load('models/model_3.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_3 = precision_score(y_train, y_pred_train_3, average='weighted')
precision_test_3 = precision_score(y_test, y_pred_test_3, average='weighted')
print('Precision (train): ', round(precision_train_3, 2))
print('Precision (test): ', round(precision_test_3, 2))

# Calculate Recall
recall_train_3 = recall_score(y_train, y_pred_train_3, average='weighted')
recall_test_3 = recall_score(y_test, y_pred_test_3, average='weighted')
print('Recall (train): ', round(recall_train_3, 2))
print('Recall (test): ', round(recall_test_3, 2))

# Calculate F1 Score
f1_train_3 = f1_score(y_train, y_pred_train_3, average='weighted')
f1_test_3 = f1_score(y_test, y_pred_test_3, average='weighted')
print('F1 Score (train): ', round(f1_train_3, 2))
print('F1 Score (test): ', round(f1_test_3, 2))

In [ ]:
# Calculate Classification Report
classification_train_3 = classification_report(y_train, y_pred_train_3)
classification_test_3 = classification_report(y_test, y_pred_test_3)
print('Classification Report (train): ', classification_train_3)
print('Classification Report (test): ', classification_test_3)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_3_train'] = round(f1_train_3, 2)
f1_scores['f2_3_test'] = round(f1_test_3, 2)

#### Model 4: Random Forest Classifier

In [ ]:
# Instantiate and train Random Forest Classifier model
model_4 = RandomForestClassifier(random_state=RSEED, max_features = 'sqrt', n_jobs=-1, verbose = 1)
model_4.fit(X_train, y_train)

##### GridSearchCV

In [ ]:
# Definition of hyperparameters
param_grid = {
    'n_estimators': [100, 200, 700],
    'max_depth': [10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2']
}

# GridSearchCV initiation
grid_search = GridSearchCV(model_4, param_grid, cv=5)

# Search for best parameters
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

param_grid = {
    'n_estimators': [100, 200, 700],
    'max_depth': [10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2']
}

Best parameters: 

In [ ]:
y_pred_train_4 = model_4.predict(X_train)
y_pred_test_4 = model_4.predict(X_test)

In [ ]:
# Save the model
dump(model_4, 'models/model_4.joblib')

In [ ]:
# Load model
model_4 = load('models/model_4.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_4 = precision_score(y_train, y_pred_train_4, average='weighted')
precision_test_4 = precision_score(y_test, y_pred_test_4, average='weighted')
print('Precision (train): ', round(precision_train_4, 2))
print('Precision (test): ', round(precision_test_4, 2))

# Calculate Recall
recall_train_4 = recall_score(y_train, y_pred_train_4, average='weighted')
recall_test_4 = recall_score(y_test, y_pred_test_4, average='weighted')
print('Recall (train): ', round(recall_train_4, 2))
print('Recall (test): ', round(recall_test_4, 2))

# Calculate F1 Score
f1_train_4 = f1_score(y_train, y_pred_train_4, average='weighted')
f1_test_4 = f1_score(y_test, y_pred_test_4, average='weighted')
print('F1 Score (train): ', round(f1_train_4, 2))
print('F1 Score (test): ', round(f1_test_4, 2))

In [ ]:
# Calculate Classification Report
classification_train_4 = classification_report(y_train, y_pred_train_4)
classification_test_4 = classification_report(y_test, y_pred_test_4)
print('Classification Report (train): ', classification_train_4)
print('Classification Report (test): ', classification_test_4)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_4_train'] = round(f1_train_4, 2)
f1_scores['f2_4_test'] = round(f1_test_4, 2)

#### Model 5: XGBClassifier

In [ ]:
# Instantiate and train XGBoost Classifier model
model_5 = XGBClassifier(random_state=RSEED)
model_5.fit(X_train, y_train - 1)

In [ ]:
y_pred_train_5 = model_5.predict(X_train)
y_pred_test_5 = model_5.predict(X_test)

In [ ]:
# Save the model
dump(model_5, 'models/model_5.joblib')

In [ ]:
# Load model
model_5 = load('models/model_5.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test - 1, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_5 = precision_score(y_train - 1, y_pred_train_5, average='weighted')
precision_test_5 = precision_score(y_test - 1, y_pred_test_5, average='weighted')
print('Precision (train): ', round(precision_train_5, 2))
print('Precision (test): ', round(precision_test_5, 2))

# Calculate Recall
recall_train_5 = recall_score(y_train - 1, y_pred_train_5, average='weighted')
recall_test_5 = recall_score(y_test - 1, y_pred_test_5, average='weighted')
print('Recall (train): ', round(recall_train_5, 2))
print('Recall (test): ', round(recall_test_5, 2))

# Calculate F1 Score
f1_train_5 = f1_score(y_train - 1, y_pred_train_5, average='weighted')
f1_test_5 = f1_score(y_test - 1, y_pred_test_5, average='weighted')
print('F1 Score (train): ', round(f1_train_5, 2))
print('F1 Score (test): ', round(f1_test_5, 2))

In [ ]:
# Calculate Classification Report
classification_train_5 = classification_report(y_train, y_pred_train_5)
classification_test_5 = classification_report(y_test, y_pred_test_5)
print('Classification Report (train): ', classification_train_5)
print('Classification Report (test): ', classification_test_5)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_5_train'] = round(f1_train_5, 2)
f1_scores['f2_5_test'] = round(f1_test_5, 2)

#### Model 6: Ada Boost

In [ ]:
# Instantiate and train Ada Boost classifier
#model_6 = AdaBoostClassifier(random_state=RSEED)
#model_6.fit(X_train, y_train)

##### RandomizedSearchCV

In [ ]:
# Define the parameter distribution for random search
param_dist = {
    'n_estimators': randint(50, 100),  # Reduced upper bound
    'learning_rate': uniform(0.01, 0.5),  # Reduced upper bound
    'base_estimator__max_depth': randint(1, 5),  # Reduced upper bound
    'base_estimator__min_samples_split': randint(2, 10),  # Reduced upper bound
    'base_estimator__min_samples_leaf': randint(1, 10),  # Reduced upper bound
    'algorithm': ['SAMME', 'SAMME.R']
}

# Create a base model
base_estimator = DecisionTreeClassifier(random_state=RSEED)
ada = AdaBoostClassifier(base_estimator=base_estimator, random_state=RSEED)

# Create a custom scorer (you can change this to other metrics if needed)
scorer = make_scorer(f1_score)

# Instantiate RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=ada,
    param_distributions=param_dist,
    n_iter=50,  # Reduced number of iterations
    cv=3,  # Reduced number of cross-validation folds
    scoring=scorer,
    random_state=RSEED,
    n_jobs=-1  # use all available cores
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Get the best model
model_6 = random_search.best_estimator_

In [ ]:
y_pred_train_6 = model_6.predict(X_train)
y_pred_test_6 = model_6.predict(X_test)

In [ ]:
# Save the model
dump(model_6, 'models/model_6.joblib')

In [ ]:
# Load model
model_6 = load('models/model_6.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_6 = precision_score(y_train, y_pred_train_6, average='weighted')
precision_test_6 = precision_score(y_test, y_pred_test_6, average='weighted')
print('Precision (train): ', round(precision_train_6, 2))
print('Precision (test): ', round(precision_test_6, 2))

# Calculate Recall
recall_train_6 = recall_score(y_train, y_pred_train_6, average='weighted')
recall_test_6 = recall_score(y_test, y_pred_test_6, average='weighted')
print('Recall (train): ', round(recall_train_6, 2))
print('Recall (test): ', round(recall_test_6, 2))

# Calculate F1 Score
f1_train_6 = f1_score(y_train, y_pred_train_6, average='weighted')
f1_test_6 = f1_score(y_test, y_pred_test_6, average='weighted')
print('F1 Score (train): ', round(f1_train_6, 2))
print('F1 Score (test): ', round(f1_test_6, 2))

In [ ]:
# Calculate Classification Report
classification_train_6 = classification_report(y_train, y_pred_train_6)
classification_test_6 = classification_report(y_test, y_pred_test_6)
print('Classification Report (train): ', classification_train_6)
print('Classification Report (test): ', classification_test_6)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_6_train'] = round(f1_train_6, 2)
f1_scores['f2_6_test'] = round(f1_test_6, 2)

#### Model 7: Bagging

In [ ]:
# Instantiate and train Bagging Classifier with base model DecisionTreeClassifier
#model_7 = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=RSEED)
#model_7.fit(X_train, y_train)

##### RandomizedSearchCV

In [ ]:
# Define the parameter distribution for random search
param_dist = {
    'n_estimators': randint(10, 200),
    'max_samples': uniform(0.5, 1.0),
    'max_features': uniform(0.1, 1.0),  
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
    'base_estimator__max_depth': randint(1, 20),
    'base_estimator__min_samples_split': randint(2, 20),
    'base_estimator__min_samples_leaf': randint(1, 20)
}

# Create a base model
base_estimator = DecisionTreeClassifier(random_state=RSEED)
bagging = BaggingClassifier(base_estimator=base_estimator, random_state=RSEED)

# Create a custom scorer (you can change this to other metrics if needed)
scorer = make_scorer(f1_score, average='weighted')  # Specify average for multi-class

# Instantiate RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=bagging,
    param_distributions=param_dist,
    n_iter=100,  # number of parameter settings that are sampled
    cv=5,  # number of cross-validation folds
    scoring=scorer,
    random_state=RSEED,
    n_jobs=-1  # use all available cores
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Get the best model
model_7 = random_search.best_estimator_

In [ ]:
y_pred_train_7 = model_7.predict(X_train)
y_pred_test_7 = model_7.predict(X_test)

In [ ]:
# Save the model
dump(model_7, 'models/model_7.joblib')

In [ ]:
# Load model
model_7 = load('models/model_7.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_7 = precision_score(y_train, y_pred_train_7, average='weighted')
precision_test_7 = precision_score(y_test, y_pred_test_7, average='weighted')
print('Precision (train): ', round(precision_train_7, 2))
print('Precision (test): ', round(precision_test_7, 2))

# Calculate Recall
recall_train_7 = recall_score(y_train, y_pred_train_7, average='weighted')
recall_test_7 = recall_score(y_test, y_pred_test_7, average='weighted')
print('Recall (train): ', round(recall_train_7, 2))
print('Recall (test): ', round(recall_test_7, 2))

# Calculate F1 Score
f1_train_7 = f1_score(y_train, y_pred_train_7, average='weighted')
f1_test_7 = f1_score(y_test, y_pred_test_7, average='weighted')
print('F1 Score (train): ', round(f1_train_7, 2))
print('F1 Score (test): ', round(f1_test_7, 2))

In [ ]:
# Calculate Classification Report
classification_train_7 = classification_report(y_train, y_pred_train_7)
classification_test_7 = classification_report(y_test, y_pred_test_7)
print('Classification Report (train): ', classification_train_7)
print('Classification Report (test): ', classification_test_7)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_7_train'] = round(f1_train_7, 2)
f1_scores['f2_7_test'] = round(f1_test_7, 2)

#### Model 8: Extra Trees

In [ ]:
# Instantiate and train extreme random tree regressor on poisson/squared_error/absolute_error/friedman_mse
model_8 = ExtraTreesClassifier(random_state=RSEED, max_features='sqrt', n_jobs=-1, verbose=1, n_estimators=200)
model_8.fit(X_train, y_train)

##### RandomizedSearchCV

In [ ]:
# # Define the parameter distribution for random search
# param_dist = {
#     #'class_weight': ['balanced', 'balanced_subsample'],  # takes count within classes into account 
#     'max_depth': randint(50, 10000),
#     'min_samples_leaf': randint(10, 100),
#     #'min_samples_split': randint(2,10000),
#     'n_estimators': randint(200, 5000),
#     'criterion': ['gini', 'entropy', 'log_loss']
# }

# # Create a base model
# XTC = ExtraTreesClassifier(random_state=RSEED, class_weight='balanced')

# # Create a custom scorer (you can change this to other metrics if needed)
# scorer = make_scorer(f1_score)

# # Instantiate RandomizedSearchCV object
# random_search = RandomizedSearchCV(
#     estimator=XTC,
#     param_distributions=param_dist,
#     n_iter=50,  # Reduced number of iterations
#     cv=3,  # Reduced number of cross-validation folds
#     scoring=scorer,
#     random_state=RSEED,
#     n_jobs=-1  # use all available cores
# )

# # Fit RandomizedSearchCV
# random_search.fit(X_train, y_train)

# # Print the best parameters and score
# print("Best parameters:", random_search.best_params_)
# print("Best cross-validation score:", random_search.best_score_)

# # Get the best model
# model_8_rscv = random_search.best_estimator_

##### GridSearchCV

In [ ]:
# # Defining parameter grid
# param_grid = {
#     'max_features': ['sqrt', 'log2', None],
#     'min_samples_leaf': [10, 100, 200, 500, 1000, 10000],
#     'n_estimators': [100, 500, 1000, 5000]
# }
    
# # Create a base model
# XTC = ExtraTreesClassifier(random_state=RSEED) #, class_weight='balanced')
    
# # Create a custom scorer (you can change this to other metrics if needed)
# scorer = make_scorer(f1_score)
    
# # Instantiate gridsearch and define the metric to optimize 
# grid_search = GridSearchCV(
#     estimator=XTC, 
#     param_grid=param_grid,
#     scoring=scorer,
#     cv=3,  # Reduced number of cross-validation folds 
#     verbose=5, 
#     n_jobs=-1 # use all available cores
# )

# # Fit GridSearchCV
# grid_search.fit(X_train, y_train)

# # Print the best parameters and score
# print("Best parameters:", grid_search.best_params_)
# print("Best cross-validation score:", grid_search.best_score_)

# # Get the best model
# model_8_gscv = grid_search.best_estimator_

In [ ]:
y_pred_train_8 = model_8.predict(X_train)
y_pred_test_8 = model_8.predict(X_test)

In [ ]:
# Save the model
dump(model_8, 'models/model_8.joblib')

In [ ]:
# Load model
model_8 = load('models/model_8.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_8 = precision_score(y_train, y_pred_train_8, average='weighted')
precision_test_8 = precision_score(y_test, y_pred_test_8, average='weighted')
print('Precision (train): ', round(precision_train_8, 2))
print('Precision (test): ', round(precision_test_8, 2))

# Calculate Recall
recall_train_8 = recall_score(y_train, y_pred_train_8, average='weighted')
recall_test_8 = recall_score(y_test, y_pred_test_8, average='weighted')
print('Recall (train): ', round(recall_train_8, 2))
print('Recall (test): ', round(recall_test_8, 2))

# Calculate F1 Score
f1_train_8 = f1_score(y_train, y_pred_train_8, average='weighted')
f1_test_8 = f1_score(y_test, y_pred_test_8, average='weighted')
print('F1 Score (train): ', round(f1_train_8, 2))
print('F1 Score (test): ', round(f1_test_8, 2))

In [ ]:
# Calculate Classification Report
classification_train_8 = classification_report(y_train, y_pred_train_8)
classification_test_8 = classification_report(y_test, y_pred_test_8)
print('Classification Report (train): ', classification_train_8)
print('Classification Report (test): ', classification_test_8)

In [ ]:
# Store classification report as pd.df

#report_train = classification_report(y_train, y_pred_train, output_dict=True)
#report_test = classification_report(y_test, y_pred_test, output_dict=True)
#df = pandas.DataFrame(report_train).transpose()
#df = pandas.DataFrame(report_test).transpose()

# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_8_train'] = round(f1_train_8, 2)
f1_scores['f1_8_test'] = round(f1_test_8, 2)

In [ ]:
f1_scores

#### Model 9: Gradient Boosting Classifier

In [ ]:
# Instantiate and train Gradient Boosting Classifier
model_9 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1.0, max_features='sqrt', random_state=RSEED)
model_9.fit(X_train, y_train)

##### RandomizedSearchCV¶

In [ ]:
# # Define the parameter distribution for random search
# param_dist = {
#     'n_estimators': randint(100, 10000),
#     'learning_rate': uniform(0.05, 1),
#     'loss': ['log_loss', 'exponential']
    
#     'class_weight': ['balanced', 'balanced_subsample'],  # takes count within classes into account 
#     'max_depth': randint(10, 10000),
#     'min_samples_leaf': randint(10, 50000),
#     #'min_samples_split': randint(2,10000),
#     'n_estimators': randint(100, 10000),
#     'criterion': ['gini', 'entropy', 'log_loss']
# }

# # Create a base model
# #base_estimator = DecisionTreeClassifier(random_state=RSEED)
# GBC = ExtraTreesClassifier(random_state=RSEED)

# # Create a custom scorer (you can change this to other metrics if needed)
# scorer = make_scorer(f1_score)

# # Instantiate RandomizedSearchCV object
# random_search = RandomizedSearchCV(
#     estimator=GBC,
#     param_distributions=param_dist,
#     n_iter=50,  # Reduced number of iterations
#     cv=3,  # Reduced number of cross-validation folds
#     scoring=scorer,
#     random_state=RSEED,
#     n_jobs=-1  # use all available cores
# )

# # Fit RandomizedSearchCV
# random_search.fit(X_train, y_train)

# # Print the best parameters and score
# print("Best parameters:", random_search.best_params_)
# print("Best cross-validation score:", random_search.best_score_)

# # Get the best model
# model_9 = random_search.best_estimator_

In [ ]:
y_pred_train_9 = model_9.predict(X_train)
y_pred_test_9 = model_9.predict(X_test)

In [ ]:
# Save the model
dump(model_9, 'models/model_9.joblib')

In [ ]:
# Load model
model_9 = load('models/model_9.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_9 = precision_score(y_train, y_pred_train_9, average='weighted')
precision_test_9 = precision_score(y_test, y_pred_test_9, average='weighted')
print('Precision (train): ', round(precision_train_9, 2))
print('Precision (test): ', round(precision_test_9, 2))

# Calculate Recall
recall_train_9 = recall_score(y_train, y_pred_train_9, average='weighted')
recall_test_9 = recall_score(y_test, y_pred_test_9, average='weighted')
print('Recall (train): ', round(recall_train_9, 2))
print('Recall (test): ', round(recall_test_9, 2))

# Calculate F1 Score
f1_train_9 = f1_score(y_train, y_pred_train_9, average='weighted')
f1_test_9 = f1_score(y_test, y_pred_test_9, average='weighted')
print('F1 Score (train): ', round(f1_train_9, 2))
print('F1 Score (test): ', round(f1_test_9, 2))

In [ ]:
# Calculate Classification Report
classification_train_9 = classification_report(y_train, y_pred_train_9)
classification_test_9 = classification_report(y_test, y_pred_test_9)
print('Classification Report (train): ', classification_train_9)
print('Classification Report (test): ', classification_test_9)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_9_train'] = round(f1_train_9, 2)
f1_scores['f1_9_test'] = round(f1_test_9, 2)

#### Model 10: Stacking

In [ ]:
# Hyperparameter grid
estimators = [
    ('sgd', model_1),
    ('knn', model_2),
    ('dt', model_3),
    ('dt', model_4),
    ('xgb', model_5),
    ('ada', model_6),
    ('bag', model_7),
    ('extra', model_8),
    ('grboo', model_9),
    ]

# Instantiate and train Stacking Classifier/Regressor model
model_10 = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier(), cv=5, stack_method='predict_proba')
model_10.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
# Save the model
dump(model_10, 'models/model_10.joblib')

In [ ]:
# Load model
model_10 = load('models/model_10.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_10 = precision_score(y_train, y_pred_train_10, average='weighted')
precision_test_10 = precision_score(y_test, y_pred_test_10, average='weighted')
print('Precision (train): ', round(precision_train_10, 2))
print('Precision (test): ', round(precision_test_10, 2))

# Calculate Recall
recall_train_10 = recall_score(y_train, y_pred_train_10, average='weighted')
recall_test_10 = recall_score(y_test, y_pred_test_10, average='weighted')
print('Recall (train): ', round(recall_train_10, 2))
print('Recall (test): ', round(recall_test_10, 2))

# Calculate F1 Score
f1_train_10 = f1_score(y_train, y_pred_train_10, average='weighted')
f1_test_10 = f1_score(y_test, y_pred_test_10, average='weighted')
print('F1 Score (train): ', round(f1_train_10, 2))
print('F1 Score (test): ', round(f1_test_10, 2))

In [ ]:
# Calculate Classification Report
classification_train_10 = classification_report(y_train, y_pred_train_10)
classification_test_10 = classification_report(y_test, y_pred_test_10)
print('Classification Report (train): ', classification_train_10)
print('Classification Report (test): ', classification_test_10)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_10_train'] = round(f1_train_10, 2)
f1_scores['f1_10_test'] = round(f1_test_10, 2)

#### Model 11: Max Voting Classifier

In [ ]:
# Instantiate models
estimators = [
    ('sgd', model_1),
    ('knn', model_2),
    ('dt', model_3),
    ('dt', model_4),
    ('xgb', model_5),
    ('ada', model_6),
    ('bag', model_7),
    ('extra', model_8),
    ('grboo', model_9),
    ]

# Instantiate and train Voting Classifier model
model_11 = VotingClassifier(eestimators=estimators, voting='soft')
model_11.fit(X_train, y_train)

In [ ]:
y_pred_train_11 = model_11.predict(X_train)
y_pred_test_11 = model_11.predict(X_test)

In [ ]:
# Save the model
dump(model_11, 'models/model_11.joblib')

In [ ]:
# Load model
model_11 = load('models/model_11.joblib')

##### Evaluation

In [ ]:
# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Create confusion matrix
conf_matrixf = pd.DataFrame(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrixf, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate Precision
precision_train_11 = precision_score(y_train, y_pred_train_11, average='weighted')
precision_test_11 = precision_score(y_test, y_pred_test_11, average='weighted')
print('Precision (train): ', round(precision_train_11, 2))
print('Precision (test): ', round(precision_test_11, 2))

# Calculate Recall
recall_train_11 = recall_score(y_train, y_pred_train_11, average='weighted')
recall_test_11 = recall_score(y_test, y_pred_test_11, average='weighted')
print('Recall (train): ', round(recall_train_11, 2))
print('Recall (test): ', round(recall_test_11, 2))

# Calculate F1 Score
f1_train_11 = f1_score(y_train, y_pred_train_11, average='weighted')
f1_test_11 = f1_score(y_test, y_pred_test_11, average='weighted')
print('F1 Score (train): ', round(f1_train_11, 2))
print('F1 Score (test): ', round(f1_test_11, 2))

In [ ]:
# Calculate Classification Report
classification_train_11 = classification_report(y_train, y_pred_train_11)
classification_test_11 = classification_report(y_test, y_pred_test_11)
print('Classification Report (train): ', classification_train_11)
print('Classification Report (test): ', classification_test_11)

In [ ]:
# Store just F1-Score (average weighted) of both train and test for this model
f1_scores['f1_11_train'] = round(f1_train_11, 2)
f1_scores['f1_11_test'] = round(f1_test_11, 2)

## Evaluation

#### Overview models and F1-Score

In [ ]:
# overall
f1_scores


In [ ]:
# per class

## Validation

## Packaging